# Pass Generator Exploration

### Some random notes:
- SGD is incredibly sensitive to step size/learning rate
- ADAM is the way to go for online models

In [19]:
from software.proto.sensor_msg_pb2 import SensorProto
import numpy as np
import matplotlib.pyplot as plt
import os
from python_tools.proto_log import ProtoLog
from python_tools.sensor_proto_log import SensorProtoLog
from shared.proto.tbots_software_msgs_pb2 import PrimitiveSet, Vision
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import math

# TODO: make this work hermetically
import sys
sys.path.append("/home/akhil/proj/Software/src/bazel-bin/python_tools/")
import cost_function_python_wrapper as cf

In [2]:
class Optimizer(object):
    def minimize():
        raise NotImplementedError("minimize not implemented")
    def approximate_gradient():
        raise NotImplementedError("approximate_gradient not implemented")

In [3]:
import numpy as np
import types

class ADAM(Optimizer):
    """Implemented based on page 2 here: https://arxiv.org/pdf/1412.6980.pdf
    """
    PAST_GRADIENT_DECAY_RATE = 0.9
    PAST_SQUARED_GRADIENT_DECAY_RATE = 0.999
    GRADIENT_APPROX_STEP_SIZE = 0.00001
    EPS = 1e-8
    
    def __init__(self, step_size: float, weights: np.array):
        self.step_size = step_size
        self.weights = weights
    
    def minimize(self, params: np.array, objective_function: types.FunctionType) -> np.array:
        past_gradient_averages = np.zeros(params.size)
        past_squared_gradient_averages = np.zeros(params.size)
        temp = params.copy()
        history = []
        
        for x in range(1, 100):
            gradient = self.approximate_gradient(temp, objective_function)
            squared_gradient = np.square(gradient)

            past_gradient_averages =\
                ADAM.PAST_GRADIENT_DECAY_RATE * past_gradient_averages +\
                                (1.0-ADAM.PAST_GRADIENT_DECAY_RATE) * gradient;
            past_squared_gradient_averages =\
                ADAM.PAST_SQUARED_GRADIENT_DECAY_RATE * past_squared_gradient_averages +\
                                (1.0-ADAM.PAST_SQUARED_GRADIENT_DECAY_RATE) * squared_gradient;
            
            bias_corrected_past_gradient_averages =\
                past_gradient_averages/(1.0-ADAM.PAST_GRADIENT_DECAY_RATE**x)
            bias_corrected_past_squared_gradient_averages =\
                past_squared_gradient_averages/(1.0-ADAM.PAST_SQUARED_GRADIENT_DECAY_RATE**x)
            
            history.append(temp)
            
            temp = temp - self.step_size * (
                bias_corrected_past_gradient_averages/(
                np.sqrt(bias_corrected_past_squared_gradient_averages) + ADAM.EPS)
            )
        return temp, history
            
        
    def approximate_gradient(self, params: np.array, objective_function: types.FunctionType):
        current_value = objective_function(params)
        next_params = params + (self.step_size * self.weights)
        next_value = objective_function(next_params)
        return (next_value - current_value)/self.step_size

In [20]:
# ADAM Sanity check on x^2

objective_function = lambda x: x**2
optimizer = ADAM(step_size=1, weights=np.array([1]))
result, history = optimizer.minimize(np.array([5]), objective_function)

x = np.linspace(-10,10,100)
y = [objective_function(a) for a in x]

# Create figure
fig = go.Figure(
    data=[go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue")),
          go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue"))],
    layout=go.Layout(
        xaxis=dict(range=[-5,5], autorange=False, zeroline=False),
        yaxis=dict(range=[0, 25], autorange=False, zeroline=False),
        title_text="ADAM on x^2 starting at x=5", hovermode="closest",
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]),
    frames=[go.Frame(
        data=[go.Scatter(
            x=[step[0]],
            y=[objective_function(step)[0]],
            mode="markers",
            marker=dict(color="red", size=10))])
        for step in history]
)

fig.show()

In [5]:
DIV_A_FIELD = cf.Field(9.0, 6.0, 1.0, 2.0, 0.18, 1.0, 0.3, 0.5);
DIV_B_FIELD = cf.Field(12.0, 9.0, 1.8, 3.6, 0.18, 1.8, 0.3, 0.5);

In [21]:
def sampleGridOfPasses(sample_x=10, sample_y=10, speed=5):
    passes = []
    x_sample = np.linspace(-4.5, 4.5, sample_x)
    y_sample = np.linspace(-3, 3, sample_y)
    for x in x_sample:
        row = []
        for y in y_sample:
            row.append(cf.Pass(cf.Point(0,0), cf.Point(x,y), speed, cf.Timestamp(1)))
        passes.append(row)
    return x_sample, y_sample, passes

In [18]:
# NOTES:
# - We need a way to start the proto logger, it shouldn't autostart (need time for setup)
# - Need a way to stop logging gracefully, closing the visualizer sometimes doesn't save the file
# - We need demux classes for all protos
primitive_set_proto_log = SensorProtoLog('/tmp/vision2/SensorProto')

yellow_team = []
blue_team = []

yellow_locations = []
blue_locations = []

for idx, vision in enumerate(primitive_set_proto_log.ssl_vision_msg): 
    if vision.HasField("detection") and len(vision.detection.balls) > 0:        
        yellow_team = cf.Team([
            cf.Robot(
                robot.robot_id,
                cf.Point(robot.x/1000, robot.y/1000),
                cf.Vector(0,0),
                cf.Angle(robot.orientation),
                cf.Angle(0),
                cf.Timestamp(vision.detection.t_capture))
            for robot in vision.detection.robots_yellow])
        
        
        blue_team = cf.Team([
            cf.Robot(
                robot.robot_id,
                cf.Point(robot.x/1000, robot.y/1000),
                cf.Vector(0,0),
                cf.Angle(robot.orientation),
                cf.Angle(0),
                cf.Timestamp(vision.detection.t_capture))
            for robot in vision.detection.robots_yellow])
        
        yellow_locations = [(robot.x/1000, robot.y/1000)
                            for robot in vision.detection.robots_yellow]
        blue_locations = [(robot.x/1000, robot.y/1000)
                            for robot in vision.detection.robots_blue]
        
        break

    
x_s, y_s, grid_of_passes = sampleGridOfPasses(10,10,1)
costs = []

for row in grid_of_passes:
    cost_row = []
    for p in row:
        cost_row.append(cf.ratePassShootScore(DIV_B_FIELD, blue_team, p))
    costs.append(cost_row)

fig = go.Figure(data=[go.Contour(z=costs, x=x_s, y=y_s),])
    
blue_x, blue_y = zip(*blue_locations)
yellow_x, yellow_y = zip(*yellow_locations)

for location in blue_locations:
    fig.add_shape(type="circle",
        xref="x", yref="y",
        fillcolor="Blue",
        x0=location[0]-0.09, y0=location[1]-0.09, 
        x1=location[0]+0.09, y1=location[1]+0.09,
        line_color="LightSeaGreen",
)

for location in yellow_locations:
    fig.add_shape(type="circle",
        xref="x", yref="y",
        fillcolor="Yellow",
        x0=location[0]-0.09, y0=location[1]-0.09, 
        x1=location[0]+0.09, y1=location[1]+0.09,
        line_color="LightYellow",
)

fig.show()

In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

# Initialize figure with 4 3D subplots
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}]])

x, y, grid_of_passes = sampleGridOfPasses(20,20)
costs = {
}

costs["ratePassShootScore"] = []

for row in grid_of_passes:
    cost_row = []
    for p in row:
        cost_row.append(cf.ratePassShootScore(DIV_B_FIELD, blue_team, p))
    costs["ratePassShootScore"].append(cost_row)

costs["ratePassEnemyRisk"] = []

for row in grid_of_passes:
    cost_row = []
    for p in row:
        cost_row.append(cf.ratePassEnemyRisk(yellow_team, p))
    costs["ratePassEnemyRisk"].append(cost_row)
    
costs["calculateInterceptRisk"] = []

for row in grid_of_passes:
    cost_row = []
    for p in row:
        cost_row.append(cf.calculateInterceptRiskTeam(yellow_team, p))
    costs["calculateInterceptRisk"].append(cost_row)
    
costs["ratePassFriendlyCapability"] = []

for row in grid_of_passes:
    cost_row = []
    for p in row:
        cost_row.append(cf.ratePassFriendlyCapability(blue_team, p, 0))
    costs["ratePassFriendlyCapability"].append(cost_row)


# adding surfaces to subplots.
fig.add_trace(
    go.Surface(x=x, y=y, z=costs["ratePassShootScore"], colorscale='Viridis', showscale=False),
    row=1, col=1)

fig.add_trace(
    go.Surface(x=x, y=y, z=costs["ratePassEnemyRisk"], colorscale='RdBu', showscale=False),
    row=1, col=2)

fig.add_trace(
    go.Surface(x=x, y=y, z=costs["calculateInterceptRisk"], colorscale='YlOrRd', showscale=False),
    row=2, col=1)

fig.add_trace(
    go.Surface(x=x, y=y, z=costs["ratePassFriendlyCapability"], colorscale='YlGnBu', showscale=False),
    row=2, col=2)

fig.update_layout(
    title_text='cost_function.h visualization',
    height=800,
    width=800
)

fig.show()